In [1]:
import pandas as pd
import glob
import numpy as np
import scipy.sparse as ss
from collections import Counter
import time
#import pyranges as pr
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,collect_list, concat, lit, when, monotonically_increasing_id, concat_ws, broadcast
import pyspark.sql.functions as F
# load in all files at the top, assert all files are correct length
# make their name the same as the input arg, minuus 'input'
spark = SparkSession.builder.master("local[*]").appName("parquet").getOrCreate()

22/09/09 18:29:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:


c = spark.read.parquet('/sbgenomics/project-files/cadd_gt20_snps_2.parquet')
print(c.count())
c.show(3)

87927481


+---------+---------+---------+----------+-----------------+----------+
|    start|reference|alternate|cadd_score|unique_variant_id|chromosome|
+---------+---------+---------+----------+-----------------+----------+
|218201127|        A|        C|      20.0|  1:218201127:A:C|         1|
|200949874|        G|        A|      20.3|  1:200949874:G:A|         1|
| 49153440|        A|        C|      20.0|   1:49153440:A:C|         1|
+---------+---------+---------+----------+-----------------+----------+
only showing top 3 rows



In [3]:
d = spark.read.parquet('/sbgenomics/project-files/cadd_gt20_snps.parquet')
print(d.count())
d.show(3)

175854962
+---------+---------+---------+----------+-----------------+----------+
|    start|reference|alternate|cadd_score|unique_variant_id|chromosome|
+---------+---------+---------+----------+-----------------+----------+
|   151503|        T|        G|      23.2|     5:151503:T:G|         5|
|139833786|        C|        G|      22.6|  5:139833786:C:G|         5|
|   151507|        T|        G|      23.2|     5:151507:T:G|         5|
+---------+---------+---------+----------+-----------------+----------+
only showing top 3 rows



In [ ]:
!rm -r /sbgenomics/project-files/cadd_gt20_snps.parquet; -y

In [4]:
e = d.drop_duplicates()
e.count()

87927481

## make sure im not finding cadd scores for high impact or eqtl variants twice. may need to filte them out of the patients gvcf aafter I find them.

### need to fiind frequency matrix prior to filtering by cadd scores. how can I filter out cadd scores from one person and not the next? the variant will have the same cadd score no matter the person


### save the variants you find in the eqtl/highImpact df for each person and then concat them all and get cadd scores all at once.

In [2]:
c = spark.read.parquet('/sbgenomics/project-files/_4_jeremy_results_aug7.parquet')
print(c.count())
c.show(3)

1177569
1177569
# c.drop('genotypes','splitFromMultiAllelic','qual','INFO_DP','var_unique_variant_id')
c.drop_duplicates(['unique_variant_id']).count()

22/08/08 19:36:53 WARN DataSource: All paths were ignored:
  file:/sbgenomics/project-files/_4_jeremy_results_aug7.parquet


1178520
+---------+---------+---------+---------+--------------------+--------------------+------+---------+---------+----------+---------+---------------------+----------------+----------+
|    start|      end|reference|alternate|   unique_variant_id|     unique_location|bucket|var_chrom|var_start|var_bucket|  var_end|var_unique_variant_id|AdditiveGenotype|chromosome|
+---------+---------+---------+---------+--------------------+--------------------+------+---------+---------+----------+---------+---------------------+----------------+----------+
|211126718|211126718|        C|<NON_REF>|1:211126718:C:<NO...|1:211126718:21112...|   207|        1|211126718|       207|211126718|      1:211126718:C:A|               0|         1|
|211128238|211128239|        G|<NON_REF>|1:211128238:G:<NO...|1:211128238:21112...|   207|        1|211128238|       207|211128238|      1:211128238:G:T|               0|         1|
|211129443|211129661|        G|<NON_REF>|1:211129443:G:<NO...|1:211129443:21112...

847058

In [36]:
c.sort('AdditiveGenotype',ascending=True).show()

+--------+--------+---------+---------+--------------------+--------------------+------+---------+---------+----------+--------+---------------------+----------------+----------+
|   start|     end|reference|alternate|   unique_variant_id|     unique_location|bucket|var_chrom|var_start|var_bucket| var_end|var_unique_variant_id|AdditiveGenotype|chromosome|
+--------+--------+---------+---------+--------------------+--------------------+------+---------+---------+----------+--------+---------------------+----------------+----------+
|33503509|33503509|        A|<NON_REF>|15:33503509:A:<NO...|15:33503509:33503509|    32|       15| 33503509|        32|33503509|      15:33503509:A:G|              -9|        15|
|33449977|33449977|        C|<NON_REF>|15:33449977:C:<NO...|15:33449977:33449977|    32|       15| 33449977|        32|33449977|      15:33449977:C:T|              -9|        15|
|32675638|32675638|        G|<NON_REF>|15:32675638:G:<NO...|15:32675638:32675638|    32|       15| 326756

In [21]:
c.where(col('start') > col('var_start')).count()

0

In [4]:
c.drop_duplicates(['var_chrom','var_start','var_end']).count()

# use var_unique_loc

1177507

In [6]:
df1 = c.groupBy(['var_chrom','var_start','var_end']).count().filter("count > 1")
df1.sort('count',ascending=False).show(truncate=False) # 

+---------+---------+---------+-----+
|var_chrom|var_start|var_end  |count|
+---------+---------+---------+-----+
|3        |195781674|195781674|4    |
|3        |195781675|195781686|4    |
|15       |20534638 |20534657 |4    |
|14       |85557828 |85557828 |4    |
|3        |195781678|195781679|4    |
|5        |181045889|181045889|4    |
|15       |20534635 |20534637 |4    |
|17       |80896428 |80896457 |4    |
|15       |20534642 |20534642 |4    |
|11       |1629998  |1629998  |4    |
|3        |195781637|195781669|4    |
|11       |1629958  |1629962  |3    |
|15       |20534621 |20534621 |3    |
|17       |79483446 |79483446 |3    |
|10       |97384397 |97384397 |3    |
|6        |69304100 |69304100 |3    |
|3        |195781619|195781627|3    |
|6        |170561963|170561965|3    |
|13       |29505458 |29505459 |3    |
|9        |91796757 |91796757 |3    |
+---------+---------+---------+-----+
only showing top 20 rows



In [7]:
c.where((col('var_chrom') == 3) &\
        (col('var_start') == 195781674) &\
        (col('var_end') == 195781674)).drop('bucket','var_bucket','reference','alternate',
                                           'chromosome','var_chrom').show()


# if the spanning row is not non_ref, do we want it?
# can we break this join into multiple 'cases'?

+---------+---------+--------------------+--------------------+---------+---------+---------------------+----------------+
|    start|      end|   unique_variant_id|     unique_location|var_start|  var_end|var_unique_variant_id|AdditiveGenotype|
+---------+---------+--------------------+--------------------+---------+---------+---------------------+----------------+
|195781592|195781687|3:195781592:GAAGA...|3:195781592:19578...|195781674|195781674|     3:195781674:A:AG|              -9|
|195781607|195781703|3:195781607:GACCT...|3:195781607:19578...|195781674|195781674|     3:195781674:A:AG|              -9|
|195781614|195781703|3:195781614:GGATG...|3:195781614:19578...|195781674|195781674|     3:195781674:A:AG|              -9|
|195781649|195781854|3:195781649:T:<NO...|3:195781649:19578...|195781674|195781674|     3:195781674:A:AG|              -9|
+---------+---------+--------------------+--------------------+---------+---------+---------------------+----------------+



In [15]:
c.drop().where(col('unique_variant_id') == '2:108483089:G:<NON_REF>').show()

+---------+---------+---------+---------+--------------------+--------------------+------+---------+---------+----------+---------+---------------------+----------------+----------+
|    start|      end|reference|alternate|   unique_variant_id|     unique_location|bucket|var_chrom|var_start|var_bucket|  var_end|var_unique_variant_id|AdditiveGenotype|chromosome|
+---------+---------+---------+---------+--------------------+--------------------+------+---------+---------+----------+---------+---------------------+----------------+----------+
|108483089|108483097|        G|<NON_REF>|2:108483089:G:<NO...|2:108483089:10848...|   106|        2|108483092|       106|108483092|      2:108483092:G:T|               0|         2|
|108483089|108483097|        G|<NON_REF>|2:108483089:G:<NO...|2:108483089:10848...|   106|        2|108483093|       106|108483093|      2:108483093:A:C|               0|         2|
|108483089|108483097|        G|<NON_REF>|2:108483089:G:<NO...|2:108483089:10848...|   106|

In [5]:
c.drop_duplicates(['unique_variant_id']).count()

847058

In [5]:
c = c.na.fill(value=-9,subset=["AdditiveGenotype"])

In [6]:
Counter(c.select('AdditiveGenotype').toPandas()['AdditiveGenotype'])

Counter({0: 1011622, 1: 91912, -9: 169972, 2: 52849})

In [10]:
c.columns

['start',
 'end',
 'reference',
 'alternate',
 'unique_variant_id',
 'unique_location',
 'bucket',
 'var_chrom',
 'var_start',
 'var_bucket',
 'var_end',
 'var_unique_variant_id',
 'AdditiveGenotype',
 'chromosome']

In [8]:
c.sample(1/1000).show(1)

+--------+--------+---------+---------+----+-------+--------------------+------+----------+----------------+----------+
|   start|     end|reference|alternate|qual|INFO_DP|   unique_variant_id|bucket|var_bucket|AdditiveGenotype|chromosome|
+--------+--------+---------+---------+----+-------+--------------------+------+----------+----------------+----------+
|32770438|32770453|        A|<NON_REF>|  90|     34|1:32770438:A:<NON...|    32|        32|               0|         1|
+--------+--------+---------+---------+----+-------+--------------------+------+----------+----------------+----------+
only showing top 1 row



In [2]:
KUTD_GF_0ZR9EDHK_geneBurdenTable.parquet

c = spark.read.parquet('/sbgenomics/project-files/CHD_BS_0302Y3N5_geneBurdenTable.parquet')
print(c.count())
c.show(3)

107822
+------+---------+-------------------+---------+-----------------+-----------+----------+------------+----------------+-----------------+----------+
|symbol|frequency|cadd_score_gene_agg|      end|unique_variant_id|       rsID|cadd_score|eqtl_boolean|AdditiveGenotype|normed_CADD_score|chromosome|
+------+---------+-------------------+---------+-----------------+-----------+----------+------------+----------------+-----------------+----------+
|CAPZA1|       19|             81.615|112709992|  1:112709992:C:A| rs41310100|     13.34|           1|               1|            1.725|         1|
|PEX11B|        4|              3.644|145654417|  1:145654417:G:A|rs188800064|     1.097|           1|               1|            0.507|         1|
|  RHOC|        6|             17.207|112687772|  1:112687772:A:G|  rs2932537|     0.024|           1|               1|            2.963|         1|
+------+---------+-------------------+---------+-----------------+-----------+----------+----------

In [5]:
#c.groupBy("symbol").count().filter("count > 1").show()
c.where(col('symbol') == 'LGALS8').show()

+------+---------+-------------------+---------+-----------------+---------+----------+------------+----------------+-----------------+----------+
|symbol|frequency|cadd_score_gene_agg|      end|unique_variant_id|     rsID|cadd_score|eqtl_boolean|AdditiveGenotype|normed_CADD_score|chromosome|
+------+---------+-------------------+---------+-----------------+---------+----------+------------+----------------+-----------------+----------+
|LGALS8|        4|             58.511|236554626|  1:236554626:T:C|rs2275687|      32.0|           0|               2|            1.683|         1|
|LGALS8|        4|             58.511|236556388|  1:236556388:A:G|rs1407518|     0.703|           1|               2|            1.683|         1|
|LGALS8|        4|             58.511|236555320|  1:236555320:T:C|rs1126627|      22.8|           0|               2|            1.683|         1|
|LGALS8|        4|             58.511|236603879| 1:236603878:AG:A|rs5781902|     3.008|           1|               2| 

In [17]:
tcsq_csvs = spark.read.option('header',True).csv('/sbgenomics/project-files/variant_annotation_csvs/')
tcsq_csvs = tcsq_csvs.drop('_c0')
tcsq_csvs = tcsq_csvs.dropDuplicates(['chromosome','start', 'reference','alternate'])
tcsq_csvs.repartitionByRange(30, "chromosome","start").write.mode('overwrite').partitionBy("chromosome")\
      .parquet("variant_annotations.parquet")
!mv variant_annotations.parquet /sbgenomics/output-files/

In [23]:
from pyspark.sql.functions import isnan, when, count, col
d.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in d.columns]).show()

+----------+-----+---+---------+---------+----+-------+-----------------+---------------+---------+---------+-------+---------------------+
|chromosome|start|end|reference|alternate|qual|INFO_DP|unique_variant_id|unique_location|var_chrom|var_start|var_end|var_unique_variant_id|
+----------+-----+---+---------+---------+----+-------+-----------------+---------------+---------+---------+-------+---------------------+
|         0|    0|  0|        0|        0|   0|      0|                0|              0|        0|        0|      0|                    0|
+----------+-----+---+---------+---------+----+-------+-----------------+---------------+---------+---------+-------+---------------------+



In [3]:
u = spark.read.parquet('/sbgenomics/project-files/CHD_all_unique_variants.parquet')
print(u.count())
u.show(3)
u.drop_duplicates(['unique_variant_id']).count()


1326355

1177569
+---------+---------+-----------------+--------------------+------------+----------+
|    start|      end|unique_variant_id|     unique_location|eqtl_boolean|chromosome|
+---------+---------+-----------------+--------------------+------------+----------+
|156286325|156286325|  1:156286325:C:T|1:156286325:15628...|           0|         1|
|156477127|156477127|  1:156477127:A:C|1:156477127:15647...|           0|         1|
|156669969|156669969|  1:156669969:C:A|1:156669969:15666...|           0|         1|
+---------+---------+-----------------+--------------------+------------+----------+
only showing top 3 rows



1326355

In [30]:
u.drop_duplicates(['unique_location']).count()

1177569

In [2]:
d = spark.read.parquet('/sbgenomics/project-files/CHD_BS_0302Y3N5_scored_variants.parquet/')
print(d.count())
d.show(3)

145593
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------------+----------+----------+------------+----------+
|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|      symbol|      rsID|cadd_score|eqtl_boolean|chromosome|
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------------+----------+----------+------------+----------+
|100285499|100285499|        G|        C|  1:100285499:G:C|1489.77|     45|                 true|[{BS_0302Y3N5, 99...|        RTCA|rs10875286|     0.417|           1|         1|
|111211500|111211500|        T|        G|  1:111211500:T:G| 101.77|      8|                 true|[{BS_0302Y3N5, 99...|RP11-96K19.4|rs12043624|     0.603|           1|         1|
|111613401|111613401|        G|        A|  1:111613401:G:A| 984.77|     30|                 true|[{BS_0

In [4]:
d = spark.read.csv('/sbgenomics/project-files/gene_regions_refseq',header=True,sep='\t')
print(d.count())
d.show(3)

172809
+----+--------------+-----+------+--------+--------+--------+--------+---------+--------------------+--------------------+-----+--------+------------+----------+--------------------+
|#bin|          name|chrom|strand| txStart|   txEnd|cdsStart|  cdsEnd|exonCount|          exonStarts|            exonEnds|score|   name2|cdsStartStat|cdsEndStat|          exonFrames|
+----+--------------+-----+------+--------+--------+--------+--------+---------+--------------------+--------------------+-----+--------+------------+----------+--------------------+
|   0|NM_001276352.2| chr1|     -|67092164|67134970|67093579|67127240|        9|67092164,67096251...|67093604,67096321...|    0|C1orf141|        cmpl|      cmpl|2,1,0,1,2,0,0,-1,-1,|
|   0|NM_001276351.2| chr1|     -|67092164|67134970|67093004|67127240|        8|67092164,67095234...|67093604,67095421...|    0|C1orf141|        cmpl|      cmpl|  0,2,1,2,0,0,-1,-1,|
|   0|   NR_075077.2| chr1|     -|67092164|67134970|67134970|67134970|       1

In [6]:
d.write.mode('overwrite')\
      .parquet("gene_regions_refseq.parquet")

In [7]:
!mv gene_regions_refseq.parquet /sbgenomics/output-files/

In [2]:

d = spark.read.parquet('/sbgenomics/project-files/tcsq_variant_gene_mappings_july13.parquet')
print(d.count())
d.show(3)

22/07/19 12:40:52 ERROR Executor: Exception in task 3.0 in stage 1.0 (TID 4) 13]
java.lang.RuntimeException: file:/sbgenomics/project-files/tcsq_variant_gene_mappings_july13.parquet/chromosome=11/part-00008-af6512ae-4e61-4bc2-8612-afa4398a2805.c000.snappy.parquet.Ab0bAf0c is not a Parquet file. expected magic number at tail [80, 65, 82, 49] but found [-43, -83, 110, -96]
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:524)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:505)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:499)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:448)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat.footerFileMetaData$lzycompute$1(ParquetFileFormat.scala:271)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat.footerFileMetaData$1(ParquetFileFormat.scala:270)
	at org.apache.spark.

Py4JJavaError: An error occurred while calling o48.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 1.0 failed 1 times, most recent failure: Lost task 3.0 in stage 1.0 (TID 4) (i-040a3037cd8c11d20 executor driver): java.lang.RuntimeException: file:/sbgenomics/project-files/tcsq_variant_gene_mappings_july13.parquet/chromosome=11/part-00008-af6512ae-4e61-4bc2-8612-afa4398a2805.c000.snappy.parquet.Ab0bAf0c is not a Parquet file. expected magic number at tail [80, 65, 82, 49] but found [-43, -83, 110, -96]
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:524)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:505)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:499)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:448)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat.footerFileMetaData$lzycomput

In [5]:
d = spark.read.parquet('/sbgenomics/project-files/CHD_BS_2829KDBN_scored_variants.parquet')
print(d.count())
d.show(3)

143484
+---------+---------+---------+---------+-----------------+------+-------+---------------------+--------------------+------+-----------+----------+------------+----------+
|    start|      end|reference|alternate|unique_variant_id|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|       rsID|cadd_score|eqtl_boolean|chromosome|
+---------+---------+---------+---------+-----------------+------+-------+---------------------+--------------------+------+-----------+----------+------------+----------+
| 10170835| 10170835|        G|        A|   3:10170835:G:A|830.77|     25|                 true|[{BS_2829KDBN, 75...|  BRK1|   rs708031|     0.439|           1|         3|
|108332657|108332657|        G|        A|  3:108332657:G:A|565.77|     38|                 true|[{BS_2829KDBN, 99...| IFT57| rs17241659|     2.075|           1|         3|
|111757169|111757169|        G|        A|  3:111757169:G:A|273.77|     25|                 true|[{BS_2829KDBN, 99...|PHLDB2|rs1485887

In [2]:
d = spark.read.parquet('/sbgenomics/project-files/find_variants_test.parquet')
print(d.count())
d.show(3)

10021
+----------+---------+---------+---------+---------+------+-------+---------------------+--------------------+--------------------+--------------------+---------+---------+---------+---------------------+
|chromosome|    start|      end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|   unique_variant_id|     unique_location|var_chrom|var_start|  var_end|var_unique_variant_id|
+----------+---------+---------+---------+---------+------+-------+---------------------+--------------------+--------------------+--------------------+---------+---------+---------+---------------------+
|         1|150755063|150755063|        G|        A|342.77|     27|                 true|[{BS_0302Y3N5, 99...|     1:150755063:G:A|1:150755063:15075...|        1|150755063|150755063|      1:150755063:G:A|
|         1|150755063|150755063|        G|<NON_REF>|342.77|     27|                 true|[{BS_0302Y3N5, 99...|1:150755063:G:<NO...|1:150755063:15075...|        1|150755063|15

In [3]:
d = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5.parquet')
print(d.count())
d.show(3)

22/07/18 12:42:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


429926698
+----------+-----+-----+-----+---------------+----------------+----+-------+---------------------+-------------------+--------------------+-------+--------------------+-----------+-----------------+----------+----------+-------+--------+-------------------+--------------+--------------+-------+---------------------+--------------------+-------------------+
|contigName|start|  end|names|referenceAllele|alternateAlleles|qual|filters|splitFromMultiAllelic|INFO_ReadPosRankSum|INFO_InbreedingCoeff|INFO_DP|INFO_ClippingRankSum|INFO_RAW_MQ|INFO_BaseQRankSum|INFO_MLEAF|INFO_MLEAC|INFO_MQ|INFO_END|INFO_HaplotypeScore|INFO_MQRankSum|INFO_ExcessHet|INFO_DS|INFO_OLD_MULTIALLELIC|           genotypes|normalizationStatus|
+----------+-----+-----+-----+---------------+----------------+----+-------+---------------------+-------------------+--------------------+-------+--------------------+-----------+-----------------+----------+----------+-------+--------+-------------------+--------------+

In [2]:

d = spark.read.parquet('/sbgenomics/project-files/KUTD_GF_0ZR9EDHK_scored_variants.parquet')
print(d.count())
d.show(3)

147779
+--------+--------+---------+---------+-----------------+-------+-------+---------------------+--------------------+-------------+---------+----------+------------+----------+
|   start|     end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|       symbol|     rsID|cadd_score|eqtl_boolean|chromosome|
+--------+--------+---------+---------+-----------------+-------+-------+---------------------+--------------------+-------------+---------+----------+------------+----------+
|10494652|10494652|        G|        T|   1:10494652:G:T| 356.77|     32|                 true|[{BS_3X3GB3T9, 99...| RP13-15M17.1| rs596537|     5.925|           1|         1|
|15359936|15359936|        A|        C|   1:15359936:A:C|1047.77|     33|                 true|[{BS_3X3GB3T9, 99...|       CLCNKB| rs515216|     0.593|           1|         1|
|16032190|16032190|        C|        G|   1:16032190:C:G|1510.77|     46|                 true|[{BS_3X3GB3T9, 99.

In [3]:

u = spark.read.parquet('/sbgenomics/project-files/KUTD_all_unique_variants.parquet')
print(u.count())
u.show(3)

472005
+---------+---------+-----------------+------------+----------+
|    start|      end|unique_variant_id|eqtl_boolean|chromosome|
+---------+---------+-----------------+------------+----------+
|100297193|100297193| 10:100297193:A:C|           0|        10|
|100791306|100791306| 10:100791306:C:G|           1|        10|
|101004334|101004334| 10:101004334:T:C|           1|        10|
+---------+---------+-----------------+------------+----------+
only showing top 3 rows



In [4]:
u.where(col('start') == col('end')).count()  # test this for the vcf?

454689

In [13]:
v = spark.read.parquet('/sbgenomics/project-files/_11_BS_0302Y3N5_chd_scored_variants.parquet')
print(v.count())
v.show(3)

22/07/20 16:34:45 WARN DataSource: All paths were ignored:
  file:/sbgenomics/project-files/_11_BS_0302Y3N5_chd_scored_variants.parquet


145593
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+-------+----------+----------+------------+----------+
|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes| symbol|      rsID|cadd_score|eqtl_boolean|chromosome|
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+-------+----------+----------+------------+----------+
|108332657|108332657|        G|        A|  3:108332657:G:A|1317.77|     39|                 true|[{BS_0302Y3N5, 99...|  IFT57|rs17241659|     2.075|           1|         3|
|111871975|111871975|        A|        T|  3:111871975:A:T|1191.77|     36|                 true|[{BS_0302Y3N5, 99...|NECTIN3| rs1282931|     4.746|           1|         3|
|114154525|114154525|        A|        G|  3:114154525:A:G|1222.77|     36|                 true|[{BS_0302Y3N5, 99...|ATP6V1A| r

In [11]:

v = v.repartition(100, "chromosome",'start')

v.rdd.getNumPartitions()

100

In [12]:
v.rdd.mapPartitions(lambda it: [sum(1 for _ in it)]).collect()

[1821,
 1782,
 1802,
 1866,
 1721,
 1775,
 1693,
 1741,
 1792,
 1826,
 1801,
 1825,
 1822,
 1807,
 1768,
 1728,
 1789,
 1827,
 1809,
 1786,
 1777,
 1807,
 1759,
 1764,
 1823,
 1784,
 1767,
 1746,
 1698,
 1811,
 1825,
 1808,
 1800,
 1745,
 1837,
 1799,
 1725,
 1814,
 1704,
 1718,
 1849,
 1827,
 1797,
 1806,
 1768,
 1837,
 1853,
 1708,
 1704,
 1751,
 1718,
 1808,
 1868,
 1857,
 1699,
 1779,
 1742,
 1812,
 1708,
 1734,
 1729,
 1767,
 1772,
 1775,
 1729,
 1836,
 1797,
 1756,
 1763,
 1739,
 1773,
 1757,
 1769,
 1767,
 1718,
 1795,
 1732,
 1705,
 1806,
 1765,
 1694,
 1739,
 1768,
 1894,
 1705,
 1760,
 1787,
 1777,
 1818,
 1801,
 1750,
 1680,
 1746,
 1870,
 1774,
 1814,
 1727,
 1697,
 1723,
 1791]

In [ ]:
v.repartitionByRange(23, "chromosome").write\
      .mode('overwrite')\
      .partitionBy("chromosome")\
      .parquet("eqtl_highImpact_scored_july8.parquet")

In [16]:
v.write.partitionBy("chromosome").parquet("temp_eqtl_highImpact_scored_july8.parquet")

In [7]:
!rm -r temp_BS_0302Y3N5.parquet

In [18]:
mv temp_eqtl_highImpact_scored_july8.parquet/  /sbgenomics/output-files/

In [5]:

a = spark.read.parquet('/sbgenomics/project-files/CHD_BS_0302Y3N5_scored_variants.parquet')
print(a.count())
a.show(3)

145593
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------------+----------+----------+------------+----------+
|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|      symbol|      rsID|cadd_score|eqtl_boolean|chromosome|
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------------+----------+----------+------------+----------+
|100285499|100285499|        G|        C|  1:100285499:G:C|1489.77|     45|                 true|[{BS_0302Y3N5, 99...|        RTCA|rs10875286|     0.417|           1|         1|
|111211500|111211500|        T|        G|  1:111211500:T:G| 101.77|      8|                 true|[{BS_0302Y3N5, 99...|RP11-96K19.4|rs12043624|     0.603|           1|         1|
|111613401|111613401|        G|        A|  1:111613401:G:A| 984.77|     30|                 true|[{BS_0

In [6]:
a.where(col('eqtl_boolean') == 0).count()

4937

In [7]:
a.drop_duplicates().count()

4998

In [6]:
Counter(list(a.select('AdditiveGenotype').toPandas()['AdditiveGenotype']) )

Counter({-9: 326, 2: 1031, 1: 3112, 0: 569})

In [3]:
r = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5.parquet')
print(r.count())
r.show(3)

22/07/08 18:04:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


429926698


+----------+-----+-----+-----+---------------+----------------+----+-------+---------------------+-------------------+--------------------+-------+--------------------+-----------+-----------------+----------+----------+-------+--------+-------------------+--------------+--------------+-------+---------------------+--------------------+-------------------+
|contigName|start|  end|names|referenceAllele|alternateAlleles|qual|filters|splitFromMultiAllelic|INFO_ReadPosRankSum|INFO_InbreedingCoeff|INFO_DP|INFO_ClippingRankSum|INFO_RAW_MQ|INFO_BaseQRankSum|INFO_MLEAF|INFO_MLEAC|INFO_MQ|INFO_END|INFO_HaplotypeScore|INFO_MQRankSum|INFO_ExcessHet|INFO_DS|INFO_OLD_MULTIALLELIC|           genotypes|normalizationStatus|
+----------+-----+-----+-----+---------------+----------------+----+-------+---------------------+-------------------+--------------------+-------+--------------------+-----------+-----------------+----------+----------+-------+--------+-------------------+--------------+----------

In [3]:

s = spark.read.parquet('/sbgenomics/project-files/_3_BS_0302Y3N5_chd_scored_variants.parquet')
print(s.count())
s.show(3)

22/07/13 13:04:32 WARN DataSource: All paths were ignored:
  file:/sbgenomics/project-files/_3_BS_0302Y3N5_chd_scored_variants.parquet


177482
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+---------+----------+------------+----------+
|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|     rsID|cadd_score|eqtl_boolean|chromosome|
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+---------+----------+------------+----------+
|151679984|151679984|        T|        A|  2:151679984:T:A|1290.77|     39|                 true|[{BS_0302Y3N5, 99...|      |         |      22.6|           0|         2|
| 30986013| 30986013|        A|        G|   2:30986013:A:G| 516.77|     45|                 true|[{BS_0302Y3N5, 99...|      |         |      20.1|           0|         2|
| 68653304| 68653304|        T|        C|   2:68653304:T:C| 399.77|     31|                 true|[{BS_0302Y3N5, 99...|PROKR1|rs4627609|   

In [20]:
print(r.where(col('eqtl_boolean') == 0).count())
print(r.where(col('eqtl_boolean') == 1).count())

4950
172532


In [17]:
r.where(col('eqtl_boolean') == 0).drop('splitFromMultiAllelic','genotypes','qual').show(4)#.dropDuplicates().count()

+--------+--------+---------+---------+-----------------+-------+------+----+----------+------------+----------+
|   start|     end|reference|alternate|unique_variant_id|INFO_DP|symbol|rsID|cadd_score|eqtl_boolean|chromosome|
+--------+--------+---------+---------+-----------------+-------+------+----+----------+------------+----------+
| 3140577| 3140577|        A|        G|    1:3140577:A:G|     40|      |    |      20.7|           0|         1|
| 3140577| 3140577|        A|        G|    1:3140577:A:G|     40|      |    |      20.7|           0|         1|
|16058547|16058547|        G|        A|   1:16058547:G:A|     72|      |    |      21.1|           0|         1|
|16058547|16058547|        G|        A|   1:16058547:G:A|     72|      |    |      21.1|           0|         1|
+--------+--------+---------+---------+-----------------+-------+------+----+----------+------------+----------+
only showing top 4 rows



In [16]:
r.where(col('eqtl_boolean') == 1).drop('splitFromMultiAllelic','genotypes','qual').show(4)#.dropDuplicates().count()

+--------+--------+---------+---------+-----------------+-------+-------------+---------+----------+------------+----------+
|   start|     end|reference|alternate|unique_variant_id|INFO_DP|       symbol|     rsID|cadd_score|eqtl_boolean|chromosome|
+--------+--------+---------+---------+-----------------+-------+-------------+---------+----------+------------+----------+
|10494652|10494652|        G|        T|   1:10494652:G:T|     47| RP13-15M17.1| rs596537|     5.925|           1|         1|
|15359936|15359936|        A|        C|   1:15359936:A:C|     28|       CLCNKB| rs515216|     0.593|           1|         1|
|16032190|16032190|        C|        G|   1:16032190:C:G|     33|RP11-169K16.8|rs6669935|     10.53|           1|         1|
|16032190|16032190|        C|        G|   1:16032190:C:G|     33|       CLCNKA|rs6669935|     10.53|           1|         1|
+--------+--------+---------+---------+-----------------+-------+-------------+---------+----------+------------+----------+


In [5]:
r.where((col('eqtl_boolean') != 0) & (col('eqtl_boolean') != 1)).count()


0

In [11]:
r.sample(1/100).show()

+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+-------------+----------+-------------------+----------+
|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|       symbol|      rsID|         cadd_score|chromosome|
+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+-------------+----------+-------------------+----------+
| 49061855| 49061855|        C|        A|   4:49061855:C:A| 622.77|     38|                 true|[{BS_0302Y3N5, 99...|             |          |               23.6|         4|
|181670782|181670782|        A|        C|  4:181670782:A:C|1411.77|     42|                 true|[{BS_0302Y3N5, 99...|             |          |               21.7|         4|
| 84038487| 84038487|        T|        C|   4:84038487:T:C| 459.77|     34|                 true|[{BS_0302Y3N5, 99...|       

In [5]:
r.where(col('unique_variant_id') == '1:6935535:A:G').show()

+-------+-------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+----------+
|  start|    end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|rsID|cadd_score|chromosome|
+-------+-------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+----------+
|6935535|6935535|        A|        G|    1:6935535:A:G|1445.77|     44|                 true|[{BS_0302Y3N5, 99...|      |    |      21.4|         1|
|6935535|6935535|        A|        G|    1:6935535:A:G|1445.77|     44|                 true|[{BS_0302Y3N5, 99...|      |    |      21.4|         1|
+-------+-------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+----------+



In [4]:
d = r.groupBy(r.columns).count().filter("count > 1")
#d.drop('count')
d.drop('splitFromMultiAllelic','qual').show()

+---------+---------+---------+---------+-----------------+-------+--------------------+------+----+----------+----------+-----+
|    start|      end|reference|alternate|unique_variant_id|INFO_DP|           genotypes|symbol|rsID|cadd_score|chromosome|count|
+---------+---------+---------+---------+-----------------+-------+--------------------+------+----+----------+----------+-----+
|  6935535|  6935535|        A|        G|    1:6935535:A:G|     44|[{BS_0302Y3N5, 99...|      |    |      21.4|         1|    2|
| 57329846| 57329846|        A|        G|   1:57329846:A:G|     47|[{BS_0302Y3N5, 99...|      |    |      20.8|         1|    2|
| 60211060| 60211060|        C|        T|  17:60211060:C:T|     51|[{BS_0302Y3N5, 35...|      |    |      24.3|        17|    2|
| 18120952| 18120952|        T|        G|  17:18120952:T:G|     24|[{BS_0302Y3N5, 99...|      |    |      22.4|        17|    2|
| 53654807| 53654807|        C|        T|   6:53654807:C:T|     44|[{BS_0302Y3N5, 99...|      |  

In [41]:
has_genes = r.where(col('symbol') != '')
no_genes = r.where(col('symbol') == '')
no_genes.count()

18419

In [2]:
gene_map = spark.read.parquet('/sbgenomics/project-files/tcsq_variant_gene_mappings_july13.parquet')
print(gene_map.count())
gene_map.show(3)

3842291
+----------+-------+---------+---------+-----------+------+
|chromosome|  start|reference|alternate|       rsID|symbol|
+----------+-------+---------+---------+-----------+------+
|        10|1041549|        T|        A|rs139754347|  IDI1|
|        10|1246418|        T|        A|rs376467079|ADARB2|
|        10|1355210|        C|      CTT| rs55913100|ADARB2|
+----------+-------+---------+---------+-----------+------+
only showing top 3 rows



In [ ]:
gene_map = spark.read.option("mode", "DROPMALFORMED").parquet('/sbgenomics/project-files/tcsq_variant_gene_mappings_july4_3.parquet')
print(gene_map.count())
gene_map.show(3)

In [37]:
gene_map = gene_map.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                                                   col("start"),lit(':'),col('reference'),
                                                   lit(':'),col('alternate')))

In [42]:
genes_added = no_genes.drop('symbol','rsID').join(gene_map.select('unique_variant_id','symbol','rsID'),['unique_variant_id'],'inner')
print(genes_added.count())
genes_added.drop('splitFromMultiAllelic').show(3)

3703


+-----------------+--------+--------+---------+---------+------+-------+--------------------+----------+----------+-------+--------+
|unique_variant_id|   start|     end|reference|alternate|  qual|INFO_DP|           genotypes|cadd_score|chromosome| symbol|    rsID|
+-----------------+--------+--------+---------+---------+------+-------+--------------------+----------+----------+-------+--------+
|   6:32828974:T:C|32828974|32828974|        T|        C|255.77|     23|[{BS_0302Y3N5, 99...|      22.6|         6|   TAP2|rs241447|
|   6:32828974:T:C|32828974|32828974|        T|        C|255.77|     23|[{BS_0302Y3N5, 99...|      22.6|         6|   TAP2|rs241447|
|   6:31636814:C:T|31636814|31636814|        C|        T|327.77|     37|[{BS_0302Y3N5, 99...|      23.5|         6|MIR6832| rs10885|
+-----------------+--------+--------+---------+---------+------+-------+--------------------+----------+----------+-------+--------+
only showing top 3 rows



In [43]:
missing_genes = no_genes.drop('symbol','rsID').join(gene_map.select('unique_variant_id','symbol','rsID'),
                                                  ['unique_variant_id'],'left_anti')
missing_genes.count()

15862

In [14]:
eqtl_highImpact_scored

In [15]:
eqtl_highImpact_scored_pandas = pd.read_csv('/sbgenomics/project-files/eqtl_highImpact_scored.parquet')
len(eqtl_highImpact_scored_pandas)

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


1205593

In [16]:
eqtl_highImpact_scored_pandas.fillna(value='None',inplace=True)
eqtl_highImpact_scored_pandas

,unique_variant_id,start,reference,alternate,rsID,symbol,eqtl_boolean,chromosome,cadd_score
0,1:100285499:G:C,100285499,G,C,rs10875286,RTCA,1,1,0.417
1,1:100285499:G:C,100285499,G,C,rs10875286,DBT,1,1,0.417
2,1:100298312:T:A,100298312,T,A,rs4908058,RTCA,1,1,6.385
3,1:100298312:T:A,100298312,T,A,rs4908058,RTCA,1,1,6.385
4,1:100298312:T:A,100298312,T,A,rs4908058,RTCA,1,1,6.385
...,...,...,...,...,...,...,...,...,...
1205588,X:49263852:G:GC,49263852,G,GC,rs200204594,LL0XNC01-36H8.1,1,X,0.410
1205589,X:76462963:AG:A,76462963,AG,A,rs201251952,MAGEE2,1,X,2.162
1205590,X:98410587:A:AC,98410587,A,AC,rs200579237,DIAPH2-AS1,1,X,0.462
1205591,Y:13160117:C:CTA,13160117,C,CTA,None,TAB3P1,0,Y,15.230


In [17]:
eqtl_highImpact_scored_pandas['chromosome'] = eqtl_highImpact_scored_pandas['chromosome'].astype(str)

eqtl_highImpact_scored = spark.createDataFrame(eqtl_highImpact_scored_pandas)

In [22]:
eqtl_highImpact_scored.count()

22/07/08 12:23:26 WARN TaskSetManager: Stage 2 contains a task of very large size (70597 KiB). The maximum recommended task size is 1000 KiB.


1188307

In [27]:
!rm -r eqtl_highImpact_scored_july8.parquet

In [21]:
# DONT WANT TO DO THIS, INDELs may not be represented in the other file
eqtl_highImpact_scored = eqtl_highImpact_scored.where(col('cadd_score') < 20.0)


In [28]:
eqtl_highImpact_scored.repartitionByRange(60, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome")\
      .parquet("eqtl_highImpact_scored_july8.parquet")

22/07/08 12:28:31 WARN TaskSetManager: Stage 12 contains a task of very large size (70597 KiB). The maximum recommended task size is 1000 KiB.
22/07/08 12:28:33 WARN TaskSetManager: Stage 13 contains a task of very large size (70597 KiB). The maximum recommended task size is 1000 KiB.


In [7]:
highImpact_eqtl_scored = spark.read.parquet('/sbgenomics/project-files/eqtl_highImpact_scored_july8.parquet')
print(highImpact_eqtl_scored.count())
highImpact_eqtl_scored.show(3)

1188307
+-----------------+---------+---------+---------+-----------+--------+------------+------------------+----------+
|unique_variant_id|    start|reference|alternate|       rsID|  symbol|eqtl_boolean|        cadd_score|chromosome|
+-----------------+---------+---------+---------+-----------+--------+------------+------------------+----------+
|  4:101774057:T:C|101774057|        T|        C| rs60252511|   NFKB1|           1|3.4930000000000003|         4|
|  4:105638442:T:A|105638442|        T|        A| rs10034172|    NPNT|           1|             1.903|         4|
|  4:105676491:A:T|105676491|        A|        T|rs189503664|EEF1A1P9|           1|             0.014|         4|
+-----------------+---------+---------+---------+-----------+--------+------------+------------------+----------+
only showing top 3 rows



In [8]:
highImpact_eqtl_scored.where(col('eqtl_boolean') == 1).count()

1184601

In [9]:
highImpact_eqtl_scored.where(col('eqtl_boolean') == 0).count()

3706

In [13]:
highImpact_eqtl_scored.count()

991688

In [4]:
plus1 = spark.read.parquet('/sbgenomics/project-files/df_scored_full_workflow_plus1.parquet')
print(plus1.count())
plus1.show(3)

5815
+----------+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+
|chromosome|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|rsID|cadd_score|
+----------+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+
|        11|120956900|120956900|        A|        G| 11:120956899:A:G|1389.77|     44|                 true|[{BS_0302Y3N5, 99...|      |    |      27.3|
|        11|120956900|120956900|        A|        G| 11:120956899:A:G|1389.77|     44|                 true|[{BS_0302Y3N5, 99...|      |    |      27.3|
|        11|130910120|130910120|        C|        T| 11:130910119:C:T| 463.77|     33|                 true|[{BS_0302Y3N5, 99...|      |    |        31|
+----------+---------+---------+---------+---------+-----------------+-------

In [5]:
reg = spark.read.parquet('/sbgenomics/project-files/df_scored_full_workflow.parquet')
print(reg.count())
reg.show(3)

5815
+----------+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+
|chromosome|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|rsID|cadd_score|
+----------+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+
|        11|120956899|120956900|        A|        G| 11:120956899:A:G|1389.77|     44|                 true|[{BS_0302Y3N5, 99...|      |    |      27.3|
|        11|120956899|120956900|        A|        G| 11:120956899:A:G|1389.77|     44|                 true|[{BS_0302Y3N5, 99...|      |    |      27.3|
|        11|130910119|130910120|        C|        T| 11:130910119:C:T| 463.77|     33|                 true|[{BS_0302Y3N5, 99...|      |    |        31|
+----------+---------+---------+---------+---------+-----------------+-------

In [12]:


gene_mappings = spark.read.parquet('/sbgenomics/project-files/tcsq_variant_gene_mappings.parquet')

AnalysisException: Path does not exist: file:/sbgenomics/project-files/tcsq_variant_gene_mappings.parquet

In [5]:
df_scored = spark.read.parquet('/sbgenomics/project-files/df_scored.parquet')
print(df_scored.count())
df_scored.show(5)

5619
+----------+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+
|chromosome|    start|      end|reference|alternate|unique_variant_id|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|rsID|cadd_score|
+----------+---------+---------+---------+---------+-----------------+-------+-------+---------------------+--------------------+------+----+----------+
|        11|120956899|120956900|        A|        G| 11:120956899:A:G|1389.77|     44|                 true|[{BS_0302Y3N5, 99...|      |    |      27.3|
|        11|120956899|120956900|        A|        G| 11:120956899:A:G|1389.77|     44|                 true|[{BS_0302Y3N5, 99...|      |    |      27.3|
|        11|130910119|130910120|        C|        T| 11:130910119:C:T| 463.77|     33|                 true|[{BS_0302Y3N5, 99...|      |    |        31|
|        11|130910119|130910120|        C|        T| 11:130910119:C:T| 463.77

In [9]:
df_scored.where((col('INFO_DP') > 25) & (col('qual') > 15))

4897

In [8]:
df_scored.where().count()

5211

In [7]:
indel_all = spark.read.parquet('/sbgenomics/project-files/cadd_gnomad_all.parquet/')
indel_all.show(1)

+--------+---------+---------+----------+--------------------+----------+
|   start|reference|alternate|cadd_score|   unique_variant_id|chromosome|
+--------+---------+---------+----------+--------------------+----------+
|36449839|        A|AAAAAAAAC|     0.513|1:36449839:A:AAAA...|         1|
+--------+---------+---------+----------+--------------------+----------+
only showing top 1 row



In [8]:

snp_gt20 = spark.read.parquet('/sbgenomics/project-files/cadd_gt20_snps.parquet')
snp_gt20.show(1)

+------+---------+---------+----------+-----------------+----------+
| start|reference|alternate|cadd_score|unique_variant_id|chromosome|
+------+---------+---------+----------+-----------------+----------+
|151503|        T|        G|      23.2|     5:151503:T:G|         5|
+------+---------+---------+----------+-----------------+----------+
only showing top 1 row



In [12]:

indels_gt20 = spark.read.parquet('/sbgenomics/project-files/cadd_gt20_gnomAD_all_chroms.parquet')
indels_gt20 = indels_gt20.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                                                   col("start"),lit(':'),col('reference'),
                                                   lit(':'),col('alternate')))

indels_gt20.write.parquet('cadd_gt20_gnomAD_all_chroms.parquet')

In [30]:
!mv eqtl_highImpact_scored_july8.parquet /sbgenomics/output-files/

In [5]:
a = spark.read.parquet('/sbgenomics/project-files/highImpact_eqtl_lt20_cadd.parquet')
a.show()


+----------+---------+---------+---------+-----------------+---------+-------+-------+---------------------+--------------------+------------------+
|chromosome|    start|reference|alternate|unique_variant_id|      end|   qual|INFO_DP|splitFromMultiAllelic|           genotypes|        CADD_score|
+----------+---------+---------+---------+-----------------+---------+-------+-------+---------------------+--------------------+------------------+
|         1|109670294|        G|        A|  1:109670294:G:A|109670295|    0.0|     43|                 true|[{BS_0302Y3N5, 99...|             2.735|
|         1|212875774|        A|        G|  1:212875774:A:G|212875775| 826.77|     19|                 true|[{BS_0302Y3N5, 57...|             2.362|
|         1|170957955|        C|        T|  1:170957955:C:T|170957956|1716.77|     40|                 true|[{BS_0302Y3N5, 99...| 6.007999999999999|
|         1|202887341|        A|        T|  1:202887341:A:T|202887342|1712.77|     46|                 tru

In [5]:
df = spark.read.parquet('/sbgenomics/project-files/cadd_gnomad_all.parquet')
df.columns

['start',
 'reference',
 'alternate',
 'cadd_score',
 'unique_variant_id',
 'chromosome']

In [4]:
df = spark.read.parquet('/sbgenomics/project-files/BS_0302Y3N5.parquet/')

df_ = df.select(['contigName','start', 'end', 'names', 'referenceAllele', \
                 'alternateAlleles', 'qual', 'INFO_DP', 'splitFromMultiAllelic','genotypes'])
print(df_.count())

df_ = df_.select(col('contigName').alias('chromosome'),'start','end',col('referenceAllele').alias('reference'),
         col('alternateAlleles').alias('alternate'),'qual','INFO_DP', 'splitFromMultiAllelic','genotypes')
df_ = df_.withColumn('alternate',F.explode('alternate'))
df_.show(1)

429926698


+----------+-----+-----+---------+---------+----+-------+---------------------+--------------------+
|chromosome|start|  end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|           genotypes|
+----------+-----+-----+---------+---------+----+-------+---------------------+--------------------+
|      chr1|10000|10007|        T|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 0,...|
+----------+-----+-----+---------+---------+----+-------+---------------------+--------------------+
only showing top 1 row



In [13]:
#df.select('genotypes').schema
#StructType(
from pyspark.sql.types import StructType,StructField, StringType,IntegerType, FloatType, BooleanType, ArrayType

genotype_subschema  =[StructField('genotypes',ArrayType(StructType([StructField('0',StringType(),'true'),
                                                                StructField('1',IntegerType(),'true'),
                                                                StructField('2',ArrayType(IntegerType(),'true'),'true'),
                                                                StructField('3',ArrayType(IntegerType(),'true'),'true'),
                                                                StructField('4',StringType(),'true'),
                                                               StructField('5',BooleanType(),'true'),
                                                               StructField('6',ArrayType(IntegerType(),'true'),'true'),
                                                               StructField('7',IntegerType(),'true'),
                                                                StructField('8',StringType(),'true'),
                                                               StructField('9',ArrayType(IntegerType(),'true'),'true'),
                                                                StructField('10',IntegerType(),'true')]),'true'),'true')]
type(a)

list

In [5]:
df_ = df_.withColumn('INFO_DP',col('genotypes').getItem(0).getItem('10'))
df_ = df_.withColumn('qual',col('genotypes').getItem(0).getItem('1'))
df_.show(4)

+----------+-----+-----+---------+---------+----+-------+---------------------+--------------------+
|chromosome|start|  end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|           genotypes|
+----------+-----+-----+---------+---------+----+-------+---------------------+--------------------+
|      chr1|10000|10007|        T|<NON_REF>|   0|      3|                false|[{BS_0302Y3N5, 0,...|
|      chr1|10007|10009|        A|<NON_REF>|   6|      7|                false|[{BS_0302Y3N5, 6,...|
|      chr1|10009|10011|        C|<NON_REF>|   9|     12|                false|[{BS_0302Y3N5, 9,...|
|      chr1|10011|10013|        C|<NON_REF>|   1|     12|                false|[{BS_0302Y3N5, 1,...|
+----------+-----+-----+---------+---------+----+-------+---------------------+--------------------+
only showing top 4 rows



In [6]:
filt = df_.where((df.INFO_DP > 15) | (df.qual > 25))
#df_noNonRef = 

#filt.where(~ (col('alternate') == '<NON_REF>')).count()


In [8]:
filt.select(filt.columns+[lit('').alias("symbol"),lit('').alias("rsID")]).show()

+----------+-----+-----+---------+---------+----+-------+---------------------+--------------------+------+----+
|chromosome|start|  end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|           genotypes|symbol|rsID|
+----------+-----+-----+---------+---------+----+-------+---------------------+--------------------+------+----+
|      chr1|13272|13273|        G|        C|  66|     22|                 true|[{BS_0302Y3N5, 66...|      |    |
|      chr1|13272|13273|        G|<NON_REF>|  66|     22|                 true|[{BS_0302Y3N5, 66...|      |    |
|      chr1|13416|13417|        C|    CGAGA|  45|     15|                 true|[{BS_0302Y3N5, 45...|      |    |
|      chr1|13416|13417|        C|<NON_REF>|  45|     15|                 true|[{BS_0302Y3N5, 45...|      |    |
|      chr1|13612|13613|        T|        A|  86|     16|                 true|[{BS_0302Y3N5, 86...|      |    |
|      chr1|13612|13613|        T|<NON_REF>|  86|     16|                 true|[{BS_0302Y3N5, 86

In [4]:
#multi_alt = df_noNonRef.where(F.size('alternateAlleles') != 1)
#multi_alt.count()

In [6]:
e = spark.read.parquet('/sbgenomics/project-files/eqtl_BS_leftsemi.parquet/')
print(e.count())
e.show(2)

284
+-------------------+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+
|  unique_variant_id|chromosome|   start|     end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|
+-------------------+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+
|chr17:74957611:A:T:|     chr17|74957611|74957612|        A|        T|473.77|     23|                 true|[{BS_0302Y3N5, 99...|
| chr2:65444642:G:C:|      chr2|65444642|65444643|        G|        C|656.77|     20|                 true|[{BS_0302Y3N5, 60...|
+-------------------+----------+--------+--------+---------+---------+------+-------+---------------------+--------------------+
only showing top 2 rows



In [11]:
#e2 = spark.read.parquet('/sbgenomics/project-files/eqtl_BS.parquet/')
#print(e2.count())
#e2.show(2)

1207976
+-----------------+-------------+--------+----------+--------+------+-------+-----------+-----------+-------+------------+--------------------+------------+----+-----------+---+---+----------------+-----------------------+--------------------+------------------+---------+----------+------------+---------+--------+---------+---------+---------+----------------------+---------+--------------+--------------+--------------------+
|          gene_id|    gene_name|gene_chr|gene_start|gene_end|strand|num_var|beta_shape1|beta_shape2|true_df|pval_true_df|          variant_id|tss_distance| chr|variant_pos|ref|alt|num_alt_per_site|rs_id_dbSNP151_GRCh38p7|minor_allele_samples|minor_allele_count|      maf|ref_factor|pval_nominal|    slope|slope_se|pval_perm|pval_beta|     qval|pval_nominal_threshold| log2_aFC|log2_aFC_lower|log2_aFC_upper|              tissue|
+-----------------+-------------+--------+----------+--------+------+-------+-----------+-----------+-------+------------+----------

## Filter on depth/quality?

In [5]:
df_noNonRef = df_.where(~ (col('alternate') == '<NON_REF>')) 
print(df_noNonRef.count())
df_noNonRef.show(5)

5760994
+----------+-----+-----+---------+---------+------+-------+---------------------+--------------------+
|chromosome|start|  end|reference|alternate|  qual|INFO_DP|splitFromMultiAllelic|           genotypes|
+----------+-----+-----+---------+---------+------+-------+---------------------+--------------------+
|      chr1|10438|10440|       AC|        A|  0.31|     12|                 true|[{BS_0302Y3N5, 26...|
|      chr1|10443|10445|       TA|        T|  0.31|     10|                 true|[{BS_0302Y3N5, 26...|
|      chr1|13272|13273|        G|        C|680.77|     22|                 true|[{BS_0302Y3N5, 66...|
|      chr1|13416|13417|        C|    CGAGA|634.73|     18|                 true|[{BS_0302Y3N5, 45...|
|      chr1|13612|13613|        T|        A| 57.77|     16|                 true|[{BS_0302Y3N5, 86...|
+----------+-----+-----+---------+---------+------+-------+---------------------+--------------------+
only showing top 5 rows



In [6]:
df_snps_sample = df_noNonRef.sample(1/1000).where((F.length("reference") == 1) & (F.length("alternate") == 1))
df_snps_sample.count()

4464

In [16]:
df_snps.dropDuplicates(['chromosome','start','end','reference','alternate']).count()

4475351

In [25]:
df_snps.dtypes

[('chromosome', 'string'),
 ('start', 'bigint'),
 ('end', 'bigint'),
 ('reference', 'string'),
 ('alternate', 'string'),
 ('qual', 'double'),
 ('INFO_DP', 'int'),
 ('splitFromMultiAllelic', 'boolean'),
 ('genotypes',
  'array<struct<0:string,1:int,2:array<int>,3:array<int>,4:string,5:boolean,6:array<int>,7:int,8:string,9:array<int>,10:int>>')]

In [7]:
cadd = spark.read.option('sep','\t').csv('/sbgenomics/project-files/cadd_chrom_files_gt20/')
cadd = cadd.select(col('_c0').alias('chromosome'),col('_c1').alias('start'),col('_c2').alias('reference'),
            col('_c3').alias('alternate'),col('_c5').alias('cadd_score'))
cadd = cadd.withColumn('chromosome',concat(lit('chr'),col('chromosome')))
cadd.show(1)

+----------+------+---------+---------+----------+
|chromosome| start|reference|alternate|cadd_score|
+----------+------+---------+---------+----------+
|     chr11|106595|        A|        C|      20.2|
+----------+------+---------+---------+----------+
only showing top 1 row



In [18]:
cadd.count()

87927481

In [8]:
cadd = cadd.withColumn("start", col('start').cast('int'))

In [29]:
cadd.dtypes

[('chromosome', 'string'),
 ('start', 'int'),
 ('reference', 'string'),
 ('alternate', 'string'),
 ('cadd_score', 'string')]

In [20]:
cadd.drop_duplicates(['chromosome','start','reference','alternate']).count()

87927481

In [ ]:
j = df_snps_sample.join(cadd,['chromosome','start','reference','alternate'],'left')
print(j.count())
j.show()

4464


In [ ]:
lst=[]
eqtl_BS_cols=eqtl_BS.columns
for i in eqtl_BS_cols:
    if eqtl_BS_cols.count(i)==2:
        ind=eqtl_BS_cols.index(i)
        lst.append(ind)
lst1=list(set(lst))
for i in lst1:eqtl_BS_cols[i]=eqtl_BS_cols[i]+'_0'
eqtl_BS=eqtl_BS.toDF(*eqtl_BS_cols)

In [1]:
!mv cadd_gnomad_all.parquet /sbgenomics/output-files/

In [35]:
cadd_indels = spark.read.option("header","true").option('sep','\t').csv('/sbgenomics/project-files/cadd_gnomad_gt_20.tsv')
print(cadd_indels.count())
cadd_indels.show(5)

273946
+------+------+------+---+--------+-----+
|#Chrom|   Pos|   Ref|Alt|RawScore|PHRED|
+------+------+------+---+--------+-----+
|     1| 16855|    TA|  T|2.374888| 22.0|
|     1| 69159|     G| GT|3.633615| 25.2|
|     1| 69469|ACAATT|  A|3.295503| 24.1|
|     1| 69745|     C| CA|2.731050| 22.9|
|     1|183128|  CCAG|  C|3.568688| 24.9|
+------+------+------+---+--------+-----+
only showing top 5 rows



In [36]:
cadd_indels = cadd_indels.select(col('#Chrom').alias('chromosome'),col('Pos').alias('start'),col('Ref').alias('reference'),
            col('Alt').alias('alternate'),col('PHRED').alias('cadd_score'))
cadd_indels.show(1)

+----------+-----+---------+---------+----------+
|chromosome|start|reference|alternate|cadd_score|
+----------+-----+---------+---------+----------+
|         1|16855|       TA|        T|      22.0|
+----------+-----+---------+---------+----------+
only showing top 1 row



In [37]:
cadd_indels.repartitionByRange(50, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome")\
      .parquet("cadd_gt20_gnomAD_all_chroms.parquet")

In [38]:
cadd_indels = spark.read.option("header","true").parquet('cadd_gt20_gnomAD_all_chroms.parquet')


In [43]:
df_indels = df_noNonRef.where(~ ((F.length("reference") == 1) & (F.length("alternate") == 1)))
df_indels.count()

1285643

## Getting High Impact and eQTL CADD scores

In [ ]:
# Need to merge High Impact and eQTL variants w/ each patients parquet gVCF file
# so we have a dataframe of each patients high Impact and eQTL variants. Then combine them 
# and then split by snp/indel

# we want to add the high impact and eqtl SNPs to the master SNPs dataframe before we merge it w/ CADD scores, 
# same thing w/ indels

In [4]:
# full snp CADD scores
chr11 = spark.read.option("header","true").parquet('/sbgenomics/project-files/cadd_chrom_files_parquet/chr11')
chr11.show(1)

+---+------+---+---+--------+-----+
|  0|     1|  2|  3|       4|    5|
+---+------+---+---+--------+-----+
| 11|426388|  A|  G|0.243482|3.628|
+---+------+---+---+--------+-----+
only showing top 1 row



In [5]:
chr12 = spark.read.option("header","true").parquet('/sbgenomics/project-files/cadd_chrom_files_parquet/chr12')
chr12.count()

399413448

In [ ]:
---------------
chr11 load took 0.08305475314458212 minutes
chr11 has : 403601226 rows
---------------
chr12 load took 0.090668519337972 minutes
chr12 has : 399413448 rows

In [6]:
# full indel CADD scores
chr11 = spark.read.option("header","true").parquet('/sbgenomics/project-files/cadd_gnomad_all.parquet')

403601226

In [7]:
# High Impact data


In [8]:
# eQTL data
eqtls = spark.read.csv('/sbgenomics/project-files/eqtls_gtexV8.csv',header=True)

eqtls_select = eqtls.select(col("chr").alias("chromosome"), col("variant_pos").alias("start"),
                             col("ref").alias("reference"),col("alt").alias("alternate"),col('gene_name'),col('rs_id_dbSNP151_GRCh38p7'))

eqtls_select = eqtls_select.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                                                   col("start"),lit(':'),col('reference'),
                                                   lit(':'),col('alternate'))) 

#eqtls_select = eqtls_select.withColumn('chromosome',F.regexp_replace(col("chromosome"), "[chr,]", "") )

#eqtls_indels = eqtls_select.where(  (F.length("reference") > 1) | (F.length("alternate") > 1)  )
#eqtls_SNPs = eqtls_select.where(  ~( (F.length("reference") > 1) | (F.length("alternate") > 1) ) )


In [15]:
df_.sample(1/10000).count()

430592

In [11]:
df_ = df_.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                                                   col("start"),lit(':'),col('reference'),
                                                   lit(':'),col('alternate'))) 

In [16]:
sample = df_.sample(1/10000)
print(sample.count())
sample.show()

43174


+----------+-------+-------+---------+---------+----+-------+---------------------+--------------------+--------------------+
|chromosome|  start|    end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|           genotypes|   unique_variant_id|
+----------+-------+-------+---------+---------+----+-------+---------------------+--------------------+--------------------+
|      chr1|  67280|  67289|        C|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 30...|chr1:67280:C:<NON...|
|      chr1| 260130| 260131|        A|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 50...|chr1:260130:A:<NO...|
|      chr1| 262220| 262229|        C|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 45...|chr1:262220:C:<NO...|
|      chr1| 362098| 362099|        C|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 27...|chr1:362098:C:<NO...|
|      chr1| 635359| 635364|        G|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 74...|chr1:635359:G:

In [17]:
eqtl_bs = sample.join(eqtls_select,['unique_variant_id'],'leftsemi')

In [ ]:
eqtl_bs.count()

In [15]:
eqtl_bs.show()

+--------------------+----------+---------+---------+---------+---------+----+-------+---------------------+--------------------+----------+-----+---------+---------+---------+-----------------------+
|   unique_variant_id|chromosome|    start|      end|reference|alternate|qual|INFO_DP|splitFromMultiAllelic|           genotypes|chromosome|start|reference|alternate|gene_name|rs_id_dbSNP151_GRCh38p7|
+--------------------+----------+---------+---------+---------+---------+----+-------+---------------------+--------------------+----------+-----+---------+---------+---------+-----------------------+
|chr10:13235225:T:...|     chr10| 13235225| 13235226|        T|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 33...|      null| null|     null|     null|     null|                   null|
|chr10:132991055:G...|     chr10|132991055|132991061|        G|<NON_REF>|null|   null|                false|[{BS_0302Y3N5, 90...|      null| null|     null|     null|     null|                   n

In [4]:

cadd_snps = spark.read.option("header","true").parquet('/sbgenomics/project-files/cadd_chrom_files_parquet/chr1')

cadd_snps.select(col('0').alias('chromosome'),col('1').alias('start'),col('2').alias('reference'),
            col('3').alias('alternate'),col('5').alias('cadd_score')).show()

+----------+------+---------+---------+------------------+
|chromosome| start|reference|alternate|        cadd_score|
+----------+------+---------+---------+------------------+
|         1|491144|        T|        C|             5.115|
|         1|491144|        T|        G|             4.758|
|         1|491145|        G|        A|              4.98|
|         1|491145|        G|        C|              4.56|
|         1|491145|        G|        T|              4.42|
|         1|491146|        C|        A|               5.9|
|         1|491146|        C|        G|             6.086|
|         1|491146|        C|        T|6.5329999999999995|
|         1|491147|        T|        A|             4.989|
|         1|491147|        T|        C| 5.531000000000001|
|         1|491147|        T|        G|             5.181|
|         1|491148|        G|        A|             4.695|
|         1|491148|        G|        C|             4.271|
|         1|491148|        G|        T|             4.12

In [8]:
cadd_snps.show(1)

+---+------+---+---+-------------------+-----+
|  0|     1|  2|  3|                  4|    5|
+---+------+---+---+-------------------+-----+
|  1|491144|  T|  C|0.37209200000000003|5.115|
+---+------+---+---+-------------------+-----+
only showing top 1 row



### Create the eQTL/High Impact dataset

In [4]:
eqtls_gtexV8_select = spark.read.option("header","true").parquet('eqtls_gtexV8_formatted.parquet')
print(eqtls_gtexV8_select.count())
#eqtls_gtexV8_select.show(2)

highImpact = spark.read.option("header","true").parquet('/sbgenomics/project-files/highImpact_noDups.parquet')
print(highImpact.count() )
#highImpact.show(3)

cols = ['chromosome', 'start', 'reference', 'alternate', 'rsID', 'symbol', 'unique_variant_id','eqtl_boolean']

1207976


13921


In [3]:
highImpact.show()

+----------+---------+---------+---------+-----------+----------+--------------------+
|chromosome|    start|reference|alternate|       rsID|    symbol|   unique_variant_id|
+----------+---------+---------+---------+-----------+----------+--------------------+
|         1|176218532|        A|        G|rs184258717|AL359265.3|1:176218532:17621...|
|        15| 51037682|        G|        C|       null| MIR4713HG|15:51037682:51037...|
|         7|123452801|        C|        T|       null|      IQUB|7:123452801:12345...|
|         7|117480144|       TC|        T|       null|      CFTR|7:117480144:11748...|
|        15| 84884752|        G|        A|       null|   SLC28A1|15:84884752:84884...|
|        17| 18778544|       AG|        A|rs201275430|    FBXW10|17:18778544:18778...|
|         2| 54359066|        C|        T|       null|   C2orf73|2:54359066:543590...|
|         1| 52375501|       GC|        G|       null|      ORC1|1:52375501:523755...|
|         8|143577353|        C|        A|r

In [13]:
eqtls_gtexV8_select = eqtls_gtexV8_select.withColumn('eqtl_boolean',lit(1))

highImpact = highImpact.withColumn('eqtl_boolean',lit(0))

In [20]:

#both = eqtls_gtexV8_select.select(cols).union(highImpact.select(cols))
#both.show(2)

#both.repartitionByRange(50, "chromosome", "start").write\
#      .mode('overwrite')\
#      .partitionBy("chromosome").parquet('eqtls_and_high_impact.parquet')
!mv eqtls_and_high_impact.parquet /sbgenomics/output-files/


In [7]:
both.where(col('eqtl_boolean') == 0).count()

13921

In [18]:
## remove cadd gt20, these will be picked

both.show() #= both.where(col('cadd_score') < 20.0)

b= spark.read.option("header","true").parquet('eqtls_and_high_impact.parquet/')
b.show()

+----------+---------+---------+---------+-----------+------------+------------------+------------+
|chromosome|    start|reference|alternate|       rsID|      symbol| unique_variant_id|eqtl_boolean|
+----------+---------+---------+---------+-----------+------------+------------------+------------+
|         1| 21265440|        A|        G|   rs785193|RP5-1071N3.1|    1:21265440:A:G|           1|
|         1| 21240588|    CAAGA|        C| rs34170976|        ECE1|1:21240588:CAAGA:C|           1|
|         1|210468094|        A|        G|   rs981754|      CAMK1G|   1:210468094:A:G|           1|
|         1|210358956|        C|        T| rs17016026|     C1orf74|   1:210358956:C:T|           1|
|         1|210223447|        G|        A| rs74156109|     SERTAD4|   1:210223447:G:A|           1|
|         1|210280593|        T|        C|   rs646512| SERTAD4-AS1|   1:210280593:T:C|           1|
|         1|210527081|        A|        G|  rs4303076|     ST13P19|   1:210527081:A:G|           1|


In [ ]:
# chr1 or 1 for chromosome column?
# patients parquet file -- chr1
# high impact -- 1
# eqtls -- 1

# cadd snps big file -- 1
# cadd indels big file -- 1

# cadd snps gt20 -- chr1
# cadd indels gt20 -- 1

In [4]:
 
eqtls_gtexV8 = spark.read.option("header","true").csv('/sbgenomics/project-files/eqtls_gtexV8.csv')
print(eqtls_gtexV8.count())
eqtls_gtexV8.show(2)

eqtls_select = eqtls_gtexV8.select(col("chr").alias("chromosome"), col("variant_pos").alias("start"),
                             col("ref").alias("reference"),col("alt").alias("alternate"),
                              col('gene_name'),col('rs_id_dbSNP151_GRCh38p7'))
eqtls_select.show(3)
eqtls_select = eqtls_select.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                                                   col("start"),lit(':'),col('reference'),
                                                   lit(':'),col('alternate')))
eqtls_select.show(2)

eqtls_select.repartitionByRange(20, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome").parquet('eqtls_gtexV8_select.parquet')

eqtls_gtexV8_select = spark.read.option("header","true").parquet('eqtls_gtexV8_select.parquet')
print(eqtls_gtexV8_select.count())
eqtls_gtexV8_select.show(2)

eqtls_FORMATTED = eqtls_gtexV8_select.withColumn('chromosome',F.regexp_replace(col("chromosome"), "[chr,]", "") )\
                    .withColumn('unique_variant_id',F.regexp_replace(col("unique_variant_id"), "[chr,]", "") )\
                    .select(['chromosome','start','reference','alternate',
                             col('rs_id_dbSNP151_GRCh38p7').alias('rsID'),
                             col('gene_name').alias('symbol'),'unique_variant_id'])
eqtls_FORMATTED.show(1)
eqtls_FORMATTED.repartitionByRange(20, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome").parquet('eqtls_gtexV8_formatted.parquet')

'''
!mv eqtls_gtexV8_select.parquet /sbgenomics/output-files/

##############################
all_indels_cadd = spark.read.parquet('/sbgenomics/project-files/cadd_gnomad_all.parquet/')
print(all_indels_cadd.count())
all_indels_cadd.show(2)

all_indels_cadd = all_indels_cadd.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                                                   col("start"),lit(':'),col('reference'),
                                                   lit(':'),col('alternate')))


all_indels_cadd.repartitionByRange(50, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome").parquet('cadd_gnomad_all.parquet')


all_indels_cadd2 = spark.read.option("header","true").parquet('cadd_gnomad_all.parquet')
print(all_indels_cadd2.count())
all_indels_cadd2.show(2)'''

22/06/23 12:55:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


1207976
+-----------------+-------------+--------+----------+--------+------+-------+-----------+-----------+-------+------------+--------------------+------------+----+-----------+---+---+----------------+-----------------------+--------------------+------------------+---------+----------+------------+---------+--------+---------+---------+---------+----------------------+---------+--------------+--------------+--------------------+
|          gene_id|    gene_name|gene_chr|gene_start|gene_end|strand|num_var|beta_shape1|beta_shape2|true_df|pval_true_df|          variant_id|tss_distance| chr|variant_pos|ref|alt|num_alt_per_site|rs_id_dbSNP151_GRCh38p7|minor_allele_samples|minor_allele_count|      maf|ref_factor|pval_nominal|    slope|slope_se|pval_perm|pval_beta|     qval|pval_nominal_threshold| log2_aFC|log2_aFC_lower|log2_aFC_upper|              tissue|
+-----------------+-------------+--------+----------+--------+------+-------+-----------+-----------+-------+------------+----------

1207976
+------+---------+---------+---------+-----------------------+-----------------+----------+
| start|reference|alternate|gene_name|rs_id_dbSNP151_GRCh38p7|unique_variant_id|chromosome|
+------+---------+---------+---------+-----------------------+-----------------+----------+
|618188|        A|        G|  FAM110C|              rs7575617|  chr2:618188:A:G|      chr2|
|242793|        G|        C|     ACP1|             rs62114506|  chr2:242793:G:C|      chr2|
+------+---------+---------+---------+-----------------------+-----------------+----------+
only showing top 2 rows



'\n!mv eqtls_gtexV8_select.parquet /sbgenomics/output-files/\n\n##############################\nall_indels_cadd = spark.read.parquet(\'/sbgenomics/project-files/cadd_gnomad_all.parquet/\')\nprint(all_indels_cadd.count())\nall_indels_cadd.show(2)\n\nall_indels_cadd = all_indels_cadd.withColumn(\'unique_variant_id\',concat(col("chromosome"), lit(":"),\n                                                   col("start"),lit(\':\'),col(\'reference\'),\n                                                   lit(\':\'),col(\'alternate\')))\n\n\nall_indels_cadd.repartitionByRange(50, "chromosome", "start").write      .mode(\'overwrite\')      .partitionBy("chromosome").parquet(\'cadd_gnomad_all.parquet\')\n\n\nall_indels_cadd2 = spark.read.option("header","true").parquet(\'cadd_gnomad_all.parquet\')\nprint(all_indels_cadd2.count())\nall_indels_cadd2.show(2)'

In [8]:
!mv eqtls_gtexV8_formatted.parquet /sbgenomics/output-files/


In [5]:
all_indels_cadd2 = spark.read.option("header","true").parquet('/sbgenomics/project-files/cadd_gnomad_all.parquet')
all_indels_cadd2.show(3)

+--------+---------+----------+----------+--------------------+----------+
|   start|reference| alternate|cadd_score|   unique_variant_id|chromosome|
+--------+---------+----------+----------+--------------------+----------+
|36449839|        A| AAAAAAAAC|     0.513|1:36449839:A:AAAA...|         1|
|36449842|        A|AAAAAAAAAC|     1.297|1:36449842:A:AAAA...|         1|
|36449843|        A|AAAAAAAAAT|     0.402|1:36449843:A:AAAA...|         1|
+--------+---------+----------+----------+--------------------+----------+
only showing top 3 rows



In [5]:
cadd_indels_gt20= spark.read.option("header","true").parquet('/sbgenomics/project-files/cadd_gt20_gnomAD_all_chroms.parquet')
cadd_indels_gt20.show(3)

+-----+---------+---------+----------+----------+
|start|reference|alternate|cadd_score|chromosome|
+-----+---------+---------+----------+----------+
|47070|        T|       TC|      23.2|        10|
|47085|        T|       TC|      23.0|        10|
|47091|        T|       TC|      23.3|        10|
+-----+---------+---------+----------+----------+
only showing top 3 rows



In [12]:
cadd_snps_gt20= spark.read.option("header","true").parquet('/sbgenomics/project-files/cadd_gt20_all_chroms.parquet')
cadd_snps_gt20.show(3)

cadd_snps_gt20 = cadd_snps_gt20.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                    col("start"),lit(':'),col('reference'),lit(':'),col('alternate')))\
                    .withColumn('chromosome',F.regexp_replace(col("chromosome"), "[chr,]", "") )\
            .withColumn('unique_variant_id',F.regexp_replace(col("unique_variant_id"), "[chr,]", "") )

cadd_snps_gt20.repartitionByRange(50, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome").parquet('cadd_gt20_snps.parquet') 

cadd_gt20_snps = spark.read.option("header","true").parquet('cadd_gt20_snps.parquet')
cadd_snps_gt20.show()

In [40]:
!mv cadd_gt20_snps.parquet /sbgenomics/output-files/


In [33]:
'''bed = spark.read.\
        option("header","true").\
        parquet('/sbgenomics/project-files/bed_protein_coding_coords_only.parquet')
bed.show(1)

whole_genome_bed = bed.withColumn('chromosome',F.regexp_replace(col("chromosome"), "[chr,]", "") )\
                        .select(col('chromosome').alias('chromosome_bed'),
                                col('start').alias('start_bed'),
                                col('end').alias('end_bed'))


whole_genome_bed.repartitionByRange(20, "chromosome_bed", "start_bed").write\
      .mode('overwrite')\
      .partitionBy("chromosome_bed").parquet('bed_protein_coding_coords_only.parquet') 

bed = spark.read.option("header","true").parquet('bed_protein_coding_coords_only.parquet')
bed.show(1)

#!mv bed_protein_coding_coords_only.parquet /sbgenomics/output-files/'''


In [5]:
bed = spark.read.option("header","true").parquet('/sbgenomics/project-files/bed_protein_coding_coords_only.parquet')


In [ ]:
# eqtls/highImpact indels columns:
# ['unique_variant_id', 'start', 'reference', 'alternate', 'cadd_score', 'chromosome']


# results_gt20 columns
['chromosome', 'start', 'end','reference', 'alternate', , 'qual', 'INFO_DP', 'splitFromMultiAllelic', 'genotypes', 
 'unique_variant_id', 'start_bed', 'end_bed', 'chromosome_bed', 'symbol', 'rsID']


# df_nonref columns:
['chromosome','start', 'end', 'reference', 'alternate', 'qual', 'INFO_DP', 'splitFromMultiAllelic', 'genotypes',
 'unique_variant_id', 'start_bed', 'end_bed', 'chromosome_bed']

# Not the right count?

In [8]:
cadd_snps_gt20.count()

175854962

# Logger for cavatica app

In [ ]:
'''
with open('stats.txt', 'w') as f:
    f.write('\n--------BED filtering-------\n')
    f.write('BED join took '+str(elapsed_bed_join/60)+' minutes\n')
    f.write('dataframe diff after bed filtering: '+str(df_cnt-cnt)+'\n')

    f.write('\n--------CADD SNP filtering-------\n')
    f.write('result_snps is '+str(result_snps.count())+ ' rows long\n')
    f.write('NonNull CADD scores: '+str(result_snps.where(col('cadd_score').isNotNull()).count())+'\n')
    f.write('join took ' +str(elapsed_snp_join/60)+ ' minutes'+'\n')

    f.write('\n--------CADD INDEL filtering-------\n')
    f.write('This patient has '+str(df_indels.count())+' indel rows\n')
    f.write('result_indels is '+str(result_indels.count())+ ' rows long\n')
    f.write('NonNull indel CADD scores: '+str(result_indels.where(col('cadd_score').isNotNull()).count())+'\n')
    f.write('join took ' +str(elapsed_indel_join/60)+ ' minutes\n')'''

In [ ]:
'''
genotype_subschema  =[StructField('genotypes',ArrayType(StructType([StructField('0',StringType(),'true'),
                                                StructField('1',IntegerType(),'true'),
                                                StructField('2',ArrayType(IntegerType(),'true'),'true'),
                                                StructField('3',ArrayType(IntegerType(),'true'),'true'),
                                                StructField('4',StringType(),'true'),
                                               StructField('5',BooleanType(),'true'),
                                               StructField('6',ArrayType(IntegerType(),'true'),'true'),
                                               StructField('7',IntegerType(),'true'),
                                                StructField('8',StringType(),'true'),
                                               StructField('9',ArrayType(IntegerType(),'true'),'true'),
                                                StructField('10',IntegerType(),'true')]),'true'),'true')]
schema = StructType([
  StructField('unique_variant_id', StringType(), True),
  StructField('start', IntegerType(), True),
  StructField('end', IntegerType(), True),
  StructField('reference', StringType(), True), 
  StructField('alternate', StringType(), True),
  StructField('qual', IntegerType(), True),
  StructField('INFO_DP', IntegerType(), True), 
  StructField('splitFromMultiAllelic',  BooleanType(), True), 
  genotype_subschema, 
  StructField('chromosome', StringType(), True), 
  ]) '''

In [ ]:
'''
with open('stats.txt', 'w') as f:
    f.write('chr11 load took '+str(elapsed_time_readAllSNPs/60)+' minutes\n')
    f.write('chr11 has : '+str(cadd_ALL_SNPs_chr11.count())+' rows')
    f.write('chr12 load took '+str(elapsed_time_readAllSNPs_12/60)+' minutes\n')
    f.write('chr12 has : '+str(cadd_ALL_SNPs_chr12.count())+' rows')
#sys.exit() '''

In [ ]:
'''print('reading in main CADD file...');  time.sleep(5); t0 = time.time()    
cadd_ALL_SNPs_chr11 = spark.read.option("header","true").parquet(input_ALL_snp_cadd_scores+'/chr11') 
print(cadd_ALL_SNPs_chr11.count())
elapsed_time_readAllSNPs = time.time() - t0
cadd_ALL_SNPs_chr12 = spark.read.option("header","true").parquet(input_ALL_snp_cadd_scores+'/chr12') 
print(cadd_ALL_SNPs_chr12.count()); elapsed_time_readAllSNPs_12 = time.time() - t0
cadd_ALL_SNPs = cadd_ALL_SNPs.select(col('0').alias('chromosome'),col('1').alias('start'),
                        col('2').alias('reference'),col('3').alias('alternate'),col('5').alias('cadd_score'))
cadd_ALL_SNPs = cadd_ALL_SNPs.withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),
                                    col("start"),lit(':'),col('reference'), lit(':'),col('alternate'))) 
print('elapsed_time to read All CADD SNPs took '+str(elapsed_time_readAllSNPs/60)+' minutes\n') '''

In [ ]:
'''  FINDING SNPs for highImpact and eqtl variants less than 20 using the big 80BG CADD file
t0 = time.time()
chroms = [row['chromosome'] for row in eqtl_highImpact_BS_snps.select('chromosome').distinct().collect()]
for n,CHROM in enumerate(chroms):
    #for n,CHROM in enumerate(['1']):
    cadd_chrom =   spark.read.parquet(input_ALL_snp_cadd_scores+'/chr'+CHROM) 
    #print('path is:  '+input_ALL_snp_cadd_scores+'/chr'+CHROM)
    #cadd_chrom = spark.read.parquet(input_cadd_snp_chr1)
    cadd_chrom = cadd_chrom.select(col("0").alias("chromosome"), col("1").alias("start"),
                    col("2").alias("reference"),col("3").alias("alternate"),col('5').alias('cadd_score'))

    cadd_chrom = cadd_chrom.withColumn('unique_variant_id',concat(col("chromosome"),
                        lit(":"),col("start"),lit(':'),col('reference'),lit(':'),col('alternate')))

    eqtls_SNPs_chrom  = eqtl_highImpact_BS_snps.where(col('chromosome') == CHROM).withColumn('unique_variant_id',concat(col("chromosome"), lit(":"),col("start"),lit(':'),col('reference'),
                                                                             lit(':'),col('alternate')))

    print(f'Joining CADD scores for chromosome {CHROM} ({eqtls_SNPs_chrom.count()} x {cadd_chrom.count()})...')
    #time.sleep(3)
    eqtls_results_chrom = eqtls_SNPs_chrom.join(cadd_chrom.select('unique_variant_id','cadd_score'), ["unique_variant_id"], "inner")

    print(f'Done...Results from chrom {CHROM} is {eqtls_results_chrom.count()} rows long.'); #time.sleep(3)

    if n == 0: eqtl_highImpact_snps_scored = eqtls_results_chrom
    else: eqtl_highImpact_snps_scored = eqtl_highImpact_snps_scored.union(eqtls_results_chrom)

t_elapsed_big_join = time.time() - t0
#eqtls_results_chrom.write.parquet('highImpact_eqtl_lt20_cadd_FULL.parquet') #sys.exit()
'''

In [ ]:
# from the (new) asd_filter_parquet tool

    
    '''
    with open('stats.txt', 'w') as f:
        f.write(f'Original df has {df_multiallelic} multiallelic rows\n')
        f.write(f'df_noNonRef df has {df_noNonRef_multiallelic} multiallelic rows\n')
        f.write(f'After join df_join has {df_join_multiallelic} multiallelic rows\n')
        f.write(f'snps has {snps_multi} multiallelic rows\n')
        f.write(f'indels has {indels_multi} multiallelic rows\n')
        f.write(f'After dedup there are {dedup_multi} multiallelic rows\n')
        
        #f.write('BED join took '+str(elapsed_bed_join/60)+' minutes\n')
        #f.write('simple joins took '+str(t_elapsed_simple_joins/60)+' minutes\n')
    #t_elapsed_save = time.time() - t0
    '''
    
    '''
    with open('stats.txt', 'w') as f:
        #f.write('Reading/formatting data took '+str(t_elapsed_read_data/60)+' minutes\n')
        #f.write('BED join took '+str(elapsed_bed_join/60)+' minutes\n')
        #f.write('simple joins took '+str(t_elapsed_simple_joins/60)+' minutes\n')
        #f.write('Main join took '+str(t_elapsed_big_join/60)+' minutes\n')
        #f.write('Concatenating took '+str(t_elapsed_concat/60)+' minutes\n')
        #f.write('Saving took '+str(t_elapsed_save/60)+' minutes\n')
        #total = t_elapsed_read_data + elapsed_bed_join +\ 
        #             t_elapsed_concat + t_elapsed_save # t_elapsed_big_join +t_elapsed_simple_joins +\
        #f.write('\nTotal runtime took '+str(total/60)+' minutes\n\n')
        
        #f.write(f'Length of df after removing NON_REFs: {dfNoNonRefLen}\n')
        f.write(f'After BED filtering we have {df_join_len} rows which consist of {df_snps.count()} SNPs and {df_indels.count()} indels.\n')
        f.write(f'Results gt20 length: {results_gt20.count()} which consist of {result_snps.count()} snps and {result_indels.count()} indels\n')
        #f.write(f'scored eqtl/hiImpact we imported length: {eqtlhiImpactLen}\n')
        f.write(f'we found {eqtl_highImpact_scored_BS.count()} eqtl/hiImpct vars in this patient (already scored).\n')
        f.write(f'after concat of results_gt20 and eqtl/HiImpact, df_scored is {df_scored.count()} long\n')
        f.write(f'and after dropping duplcates variants, its {df_scored_dedup.count()} long')
    
    '''



    '''
    ### Load in scored daata here for testing and skip everything above ####
    print('Reading in df_scored dataframe...')
    df_scored = spark.read.option("header","true").parquet(input_fake_df_scored)

    df_scored_filt = df_scored.where((col('INFO_DP') > 25) & (col('qual') > 15))
    
    dfp = df_scored_filt.toPandas()
    
    symbol_freqs = dict(Counter(dfp['symbol']))
    symbol_freq_df = pd.DataFrame.from_dict(symbol_freqs,orient='index',columns=['frequency']) 
    symbol_freq_df.index.name = 'symbol'
    
    
    gene_scores = dfp[['symbol','CADD_score']].groupby('symbol').sum()
    print(gene_scores.head())
    #df_NonRef = df_NonRef.select(df_NonRef.columns+[lit('').alias("symbol"),lit('').alias("rsID"),lit('').alias("cadd_score")]) # create null cols here so we can concat w/ the no-nonref df
    #df = df_scored.union(df_NonRef.select(c))
    '''
    
    

    '''
    print('###############################\n#### Beginning Phase Two #####\n###############################')
    
    #df = df_join; 
    
    bed = whole_genome_bed; time.sleep(5)
    
    bs_name = df.withColumn('bs_id',df.genotypes.getItem(0).getItem('0')).select('bs_id').take(1)[0][0]
    df = df.withColumn('calls', df.genotypes.getItem(0).getItem('6'))
    df = df.withColumn('calls_str',concat(df.calls.getItem(0), lit(":"), df.calls.getItem(1) ))
    df = df.withColumn('AdditiveGenotype',when(df.calls_str.contains('-')  ,-9)\
                          .otherwise(df.calls.getItem(0) + df.calls.getItem(1)))
    
    # Create Participant ID column (should just be one participant per file)
    df = df.withColumn('participant_id', df.genotypes.getItem(0).getItem('0'))
    df = df.withColumn('INFO_DP',df.genotypes.getItem(0).getItem('10'))
    df = df.withColumn('qual',df.genotypes.getItem(0).getItem('1'))
    df = df.withColumn('PL',df.genotypes.getItem(0).getItem('9'))
    
    select_cols = ['chromosome','start','end','participant_id','unique_variant_id','reference','alternate','calls','AdditiveGenotype','splitFromMultiAllelic','PL',  'qual','INFO_DP']
    
    print('df columns are already filtered correctly? --',str(set(df.columns) == select_cols))
    df = df.select(select_cols)  # need this?
    print('Finished creating starter cols'); time.sleep(3)
    
    df = df.withColumn('AdditiveGenotype',when((df.INFO_DP < 15) | (df.qual < 25) ,-9)\
                          .otherwise(df.AdditiveGenotype))
                          
    print('Labeled low qual variants...'); time.sleep(3)
    
    df = df.where(~ ((df.splitFromMultiAllelic == 'true') & (df.alternate == '<NON_REF>') ))
    
    print('Sorting...'); df = df.sort(['chromosome','start']); print('Done sorting'); time.sleep(2)
    
    df = df.drop(*("splitFromMultiAllelic")).dropDuplicates(['unique_variant_id','AdditiveGenotype'])

    print('Labeling ambiguous PL scored variants as unknown...'); time.sleep(2)
    df = df.withColumn('PL_00',df.PL.getItem(0))
    df = df.withColumn('PL_01',df.PL.getItem(1))
    df = df.withColumn('PL_11',df.PL.getItem(2))
    df = df.select(['unique_variant_id','chromosome','start','end','reference','alternate','PL','PL_00',
                                   'PL_01','PL_11','AdditiveGenotype'])
    
    df = df.withColumn('AdditiveGenotype',when((df.PL_00 == 0) & (df.PL_01 == 0) &\
                                               (col('AdditiveGenotype') == 0 ),-9).otherwise(df.AdditiveGenotype))
    
    df = df.withColumn('AdditiveGenotype',when(((df.PL_00 == 0) & (df.PL_01 == 0) &\
                         (df.PL_11 == 0)) & (col('AdditiveGenotype') == 0 ) ,-9).otherwise(df.AdditiveGenotype))
                                                  
    df = df.drop(*("PL","PL_00","PL_01","PL_11"))
    
    ########################################################
    bed = bed.rdd.map(lambda x: (x['start_bed'],x['end_bed'],
                                        x['chromosome_bed'],list(range(x['start_bed'],x['end_bed']))))


    print('Converting RDD to DF...');time.sleep(3)
    bed = bed.toDF(['start_bed','end_bed','chromosome_bed','expanded_int'])
    print(bed.columns)
    bed.write.mode('overwrite').parquet("bed_expanded_gvcf.parquet")

    
    #for n,row in bed.iterrows():
    #    exp_int = list(range(row['start_bed'],row['end_bed']))
    #    expanded_gene_intervals.append(exp_int)
    #    unique_id = row['chromosome_bed'] +':'+ pd.Series(exp_int).astype(str) +':'+\
    #                                   pd.Series([i + 1 for i in exp_int]).astype(str)
    #    genes_unq_var_ids.append(unique_id.values)
    '''




'''
print('###############################\n#### Beginning Phase Three #####\n###############################')

print('\n\n\nStarting expansion process...\n\n\n')

def flatten_list(ls,end=False):
    if end: return [str(int(item) + 1) for sublist in ls for item in sublist]
    else: return [str(item) for sublist in ls for item in sublist]
    
df = df.toPandas()

df['extra_info'] =[[i,j,k] for i,j,k in zip(df['referenceAllele'], df['alternateAlleles'], df['AdditiveGenotype'])]
df['expand_bool'] =(df['start']!=df['end'])  & (df['start']!=df['start']+1) &\
                    (df['AdditiveGenotype'] != 1) & (df['AdditiveGenotype'] != 2)


cols = ['unique_variant_id', 'chromosome', 'start', 'end', 'AdditiveGenotype'] 

missing_variants = []
unknown_GTs = []

chroms = [row['chromosome'] for row in eqtl_highImpact_BS_snps.select('chromosome').distinct().collect()]

for CHROM in bed.chrom.unique(): 
    t0=  time.time()

    df_chrom = df[df['contigName'] == CHROM]   # df.where(col('chromosome') == CHROM)
    df_chrom_filt = df_chrom[df_chrom['expand_bool']]   # df_chrom_filt = df_chrom.where(col('expanded_bool') != 1)

    locations_to_add = []
    extra_info = []
        
    for row in df_chrom_filt.itertuples():
        locations_to_add.extend(list(range(int(row[3]),int(row[4]))))
        extra_info.append(row[8] + [int(row[4]) - int(row[3])]) 

    df_new_pos = pd.DataFrame(locations_to_add); df_new_pos.columns = ['start']
    df_new_pos['end'] = [int(i)+1 for i in locations_to_add] 
    df_new_pos['AdditiveGenotype'] = flatten_list([[i[2]]*i[3] for i in extra_info])
    df_new_pos['referenceAllele'] = flatten_list([[i[0]]*i[3] for i in extra_info])
    df_new_pos['alternateAlleles'] = flatten_list([[i[1]]*i[3] for i in extra_info])
    df_new_pos['contigName'] = CHROM
    
    df_new_pos['unique_variant_id'] =  df_new_pos['chromosome'] +":"+ df_new_pos['start'].astype(str) +":"+ df_new_pos['end'].astype(str) +":"+\
                    df_new_pos['reference'] +":"+ df_new_pos['alternate']


    df_chrom_expanded = pd.concat([df_chrom[cols], df_new_pos[cols]]) 

    unknown_GTs.extend(df_chrom_expanded[df_chrom_expanded['AdditiveGenotype'] == -9]['unique_variant_id'].values)

    df_chrom_expanded['id2match']= [':'.join(i[:3]) for i in df_chrom_expanded['unique_variant_id'].str.split(':')]    
    missing = set(np.concatenate(bed[bed['chrom'] == CHROM].unique_variant_id.values)) - set(df_chrom_expanded['id2match'])
    
    missing_variants.extend(list(missing))
    print(f'Found {len(missing)} missing variants in ',np.round(time.time()-t0,2),f' seconds for {CHROM}')
    


print('Found a total of',len(missing_variants),'missing variants.') 
print('Found a total of',len(unknown_GTs),'unknown variants.')

assert 0 == len(set(missing_variants).intersection(set(df_chrom['unique_variant_id'])))
print('Creating dfs and joining them...')

df_missing = pd.DataFrame(missing_variants,columns=['unique_variant_id'])
df_missing['Chromosome'] = [i.split(':')[0] for i in df_missing['unique_variant_id'] ]
df_missing['Start'] = [i.split(':')[1] for i in df_missing['unique_variant_id'] ]
df_missing['End'] = [i.split(':')[2] for i in df_missing['unique_variant_id'] ]
df_missing['AdditiveGenotype'] = -9

df_unknown = pd.DataFrame(unknown_GTs,columns=['unique_variant_id'])
df_unknown['Chromosome'] = [i.split(':')[0] for i in df_unknown['unique_variant_id'] ]
df_unknown['Start'] = [i.split(':')[1] for i in df_unknown['unique_variant_id'] ]
df_unknown['End'] = [i.split(':')[2] for i in df_unknown['unique_variant_id'] ]
df_unknown['AdditiveGenotype'] = -9

df_missing = df_missing.drop_duplicates()
df_unknown = df_unknown.drop_duplicates()

df_1_2s = df[ (df['AdditiveGenotype'] == 1) | (df['AdditiveGenotype'] == 2)]\
                    [['unique_variant_id','contigName','start','end','AdditiveGenotype']]

df_1_2s = df_1_2s.rename(columns={'contigName': 'Chromosome', 'start': 'Start','end':'End'})

df_result = pd.concat([df_missing,df_unknown, df_1_2s]).sort_values(['Chromosome','Start'])
'''


In [ ]:
# From the find_variants tool

'''
# Find variants that we already have genotypes for 
# must use entire df, bc we are matching locations here not variants
t0  = time.time()
print('Finding direct matches...'); #time.sleep(3)
direct_matches = variants_noneqtl.join(df,['unique_location'],'inner')  # .select('unique_location','unique_variant_id','AdditiveGenotype')
dmatch_cnt = direct_matches.count()
print(f'Direct matches found: {dmatch_cnt}');time.sleep(5)
t_direct_matches = time.time() - t0

direct_matches_dedp = direct_matches.dropDuplicates()
#print('Dups: ', direct_matches.count(),direct_matches.dropDuplicates().count()); time.sleep(3)

print('Getting list of locations from direct match results'); time.sleep(3)
t=time.time()
uniq_locs = [row['unique_location'] for row in direct_matches_dedp.select('unique_location').collect()]
t_direct_locs = time.time() - t0

with open('stats.txt', 'w') as f:
    f.write('Removing eqtls took '+str(t_var_noneqtl/60)+' minutes\n')
    f.write('Finding direct matches took '+str(t_var_noneqtl/60)+' minutes (largest join)\n')
    f.write(f'Found {vnon_cnt} non-eqtls out of {variants.count()} variants\n')
    f.write(f'Found {dmatch_cnt} direct matches out of {vnon_cnt} variants\n')
    #f.write('first part took '+str(tend/60)+' minutes\n')
    f.write('Dups: ' + str(dmatch_cnt) + str(direct_matches_dedp.count()))
    f.write(f'Extracting direct match locations took: {t_direct_locs/60}')
    #f.write('Variants file # of partitions: ',str(variants.rdd.getNumPartitions()))
'''  

In [ ]:
# from the (new) asd_parquet_filter tool

'''
print('Joining eQTLs/highImpact variants with patient variants...',end=''); #time.sleep(2)
eqtl_highImpact_BS = eqtls_high_impact.join(df_noNonRef.drop('start','chromosome','reference','alternate'),['unique_variant_id'],'inner')
print(f'Found {eqtl_highImpact_BS.count()} eQTL/HighImpacts variations in this patients data.');#time.sleep(3)

eqtl_highImpact_BS_snps = eqtl_highImpact_BS.where(((F.length("reference")==1) & (F.length("alternate") == 1)))
eqtl_highImpact_BS_indels= eqtl_highImpact_BS.where(~((F.length("reference")==1)& (F.length("alternate")== 1)))
print(f'For eQTL/High_Impacts: Found {eqtl_highImpact_BS_snps.count()} SNPs and {eqtl_highImpact_BS_indels.count()} indels in this patients variants');# time.sleep(3)

############# Fetch CADD scores for the eQTLs and High Impact Variants ##################
#if eqtl_highImpact_BS_indels.count() > 0:
eqtl_highImpact_indels_scored = eqtl_highImpact_BS_indels.join(cadd_ALL_indels.select('unique_variant_id','cadd_score'),['unique_variant_id'],'inner') 
print('Found '+str(eqtl_highImpact_indels_scored.count())+' eQTLs in the CADD indels list');# time.sleep(5)  
#else:eqtl_highImpact_indels_scored = spark.createDataFrame([], schema)
t_elapsed_simple_joins = time.time() - t0 '''

#result_indels = result_indels.select(result_indels.columns+[lit('').alias("symbol"),lit('').alias("rsID")])
#assert eqtl_highImpact_snps_scored.columns == eqtl_highImpact_indels_scored.columns
#assert set(results_gt20.columns) == set(eqtl_highImpact_scored_BS.columns)


In [ ]:

    '''
    df_join = df_join.withColumn('calls', df_join.genotypes.getItem(0).getItem('6'))
    df_join = df_join.withColumn('calls_str',concat(df_join.calls.getItem(0), lit(":"), df_join.calls.getItem(1) ))
    df_join = df_join.withColumn('AdditiveGenotype',when(df_join.calls_str.contains('-')  ,-9)\
                          .otherwise(df_join.calls.getItem(0) + df_join.calls.getItem(1)))
    
    # Create Participant ID column (should just be one participant per file)
    df_join = df_join.withColumn('participant_id', df_join.genotypes.getItem(0).getItem('0'))
    df_join = df_join.withColumn('INFO_DP',df_join.genotypes.getItem(0).getItem('10'))
    df_join = df_join.withColumn('qual',df_join.genotypes.getItem(0).getItem('1'))
    df_join = df_join.withColumn('PL',df_join.genotypes.getItem(0).getItem('9'))

    df_join = df_join.withColumn('AdditiveGenotype',when((df_join.INFO_DP < 15) | (df_join.qual < 25) ,-9)\
                          .otherwise(df_join.AdditiveGenotype))
    df_join = df_join.where(~ ((df_join.splitFromMultiAllelic == 'true') & (df_join.alternate == '<NON_REF>') ))
    
    #df_join = df_join.sort(['chromosome','start'])
    #df_join = df_join.drop(*("splitFromMultiAllelic")).dropDuplicates(['unique_variant_id','AdditiveGenotype'])
    
    df_join = df_join.withColumn('PL_00',df_join.PL.getItem(0)).withColumn('PL_01',df_join.PL.getItem(1)).withColumn('PL_11',df_join.PL.getItem(2))

    df_join = df_join.withColumn('AdditiveGenotype',when((df_join.PL_00 == 0) & (df_join.PL_01 == 0) &\
                                               (col('AdditiveGenotype') == 0 ),-9).otherwise(df_join.AdditiveGenotype))
                                               
    df_join = df_join.withColumn('AdditiveGenotype',when(((df_join.PL_00 == 0) & (df_join.PL_01 == 0) &\
                         (df_join.PL_11 == 0)) & (col('AdditiveGenotype') == 0 ) ,-9).otherwise(df_join.AdditiveGenotype))
    
    df_join = df_join.drop(*("PL","PL_00","PL_01","PL_11"))
    
    # Maybe drop all -9's bc if its not found its -9 anyway? what % of -9's do we have?
    df_cnt2 = df_join.count()
    '''